In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.dates as mdates
from datetime import datetime
import matplotlib.pyplot as plt
import glob as gb
import os
import matplotlib.dates as mdates

%matplotlib notebook

In [ ]:
# Load HDF file

# df = pd.HDFStore('../Data/analysis_10mins.h5')['df']
# df = pd.HDFStore('../Data/analysis_T6.h5')['df']
df = pd.HDFStore('../Data/analysisAA.h5')['df']
# df = pd.HDFStore('../Data/analysis_flipflop.h5')['df']

# Create column for time since start of measurement
df['delta'] = (df['datetime']-df['datetime'][0]).fillna(0).astype('timedelta64[us]')/(1E6*60)

# df.info()

In [ ]:
# Get some useful data for plotting later
flush_time = df['run_time'][0]

In [ ]:
df.head()
# df.info()

# Real time raw data

In [ ]:
# Create plot of lifetime vs time
fig, ax = plt.subplots()
ax.plot(df['datetime'], df['tau'], 'o', alpha=0.3)
# ax.plot(df['datetime'], df['tempC'], '-')

# format the ticks
# ax.xaxis.set_major_locator(mdates.HourLocator())
# ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(mdates.MinuteLocator(byminute=np.arange(0, 60, flush_time)))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
fig.autofmt_xdate()  # Rotate slightly

ax.xaxis.grid(True, which="major")
ax.yaxis.grid()
plt.tight_layout()
plt.ticklabel_format(useOffset=False, axis='y')

plt.xlabel('Time (H:M)')
plt.ylabel('Lifetime (ms)')
# plt.savefig("Data/" + str(timestamp) + '/lifetimeVsTime.png', dpi=500)

In [ ]:
# Create plot of lifetime vs time
fig, ax = plt.subplots()
ax.plot(df['datetime'], df['thermocouple_in'], '-')
ax.plot(df['datetime'], df['thermocouple_out'], '-')
# ax.plot(df['datetime'], df['tempC'], '-')

# format the ticks
# ax.xaxis.set_major_locator(mdates.HourLocator())
# ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(mdates.MinuteLocator(byminute=np.arange(0, 60, flush_time)))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
fig.autofmt_xdate()  # Rotate slightly

ax.xaxis.grid(True, which="major")
ax.yaxis.grid()
plt.tight_layout()
plt.ticklabel_format(useOffset=False, axis='y')

plt.xlabel('Time (H:M)')
plt.ylabel('Lifetime (ms)')
plt.legend()
# plt.savefig("Data/" + str(timestamp) + '/lifetimeVsTime.png', dpi=500)

# Rolling Mean

In [ ]:
r = df.set_index('delta').rolling(window=100)

fig, ax = plt.subplots()

# ax.plot(df['delta'], df['tau'], 'o', alpha=0.3)
ax.plot(r['tau'].mean(), '-', lw=2)

ax.xaxis.grid(True, which="major")
ax.yaxis.grid()
plt.xticks(np.arange(0, max(df['delta'])+1, flush_time))
# plt.xticks(np.arange(0, max(df['delta'])+1, 5))
# plt.tight_layout()
plt.ticklabel_format(useOffset=False, axis='y')

# plt.title('T6: New sample every {:d} mins'.format(flush_time))
plt.xlabel('Time since start of experiment (Mins)')
plt.ylabel('Lifetime (ms)')
# plt.savefig("../Data/RollingMean.png", dpi=600)

# Drop Sweeps

In [ ]:
# Drop first 600 sweeps (300 sweeps = 1 min @ 5HZ capture rate)
df = df[df.sweep_no > 900]
# df = df[df.sweep_no < 900]

# Each concentration vs lifetime

In [ ]:
# All in one
fig, ax = plt.subplots()

for key, grp in df.groupby(['concentration']):
    ax.plot(grp['delta'], grp['tau'], 'o', alpha=0.3, label=key)
    grp['rolling'] = grp['tau'].rolling(window=50).mean()
    ax.plot(grp['delta'], grp['rolling'], '-', lw=2)  # label='rolling ({k})'.format(k=key)

ax.xaxis.grid(True, which="major")
plt.xticks(np.arange(0, max(df['delta'])+1, flush_time))
plt.legend(loc='best', ncol=3, fontsize='small')

In [ ]:
# Individual plots
for key, grp in df.groupby(['concentration']):
    plt.figure()
    plt.plot(grp['delta'], grp['tau'], 'o', alpha=0.3, label=key)
    grp['rolling'] = grp['tau'].rolling(window=50).mean()
    plt.plot(grp['delta'], grp['rolling'], '.', lw=3, color='red', label='rolling ({k})'.format(k=key)) 
    plt.title(key)
    plt.grid()
    plt.legend()
#     pass

## tau & A vs concentration

In [ ]:
grouped = df.groupby(['concentration'])
mean = grouped.mean()
stdd = grouped.std()

mean.head()

In [ ]:
stdd.head()

In [ ]:
mean = mean.reset_index()
stdd = stdd.reset_index()

fig, ax1 = plt.subplots()

ax2 = ax1.twinx()
ax1.errorbar(mean['concentration'], mean['tau'], yerr=stdd['tau'], color='b', lw='2')
ax2.errorbar(mean['concentration'], mean['A'], yerr=stdd['A'], color='g', lw='2')

ax1.set_xlabel('Glucose concentration (% Weight)')
ax1.set_ylabel('Lifetime (ms)', color='b')
ax2.set_ylabel('Amplitude (A.U.)', color='g')

ax1.grid()
plt.tight_layout()
ax1.ticklabel_format(useOffset=False, axis='y')
ax2.ticklabel_format(useOffset=False, axis='y')

plt.tight_layout()
# plt.title(df['chip'][0])
plt.savefig("../Data/Plots/together_T6" + '.png', dpi=600)

In [ ]:
fig, ax1 = plt.subplots()

ax2 = ax1.twinx()
ax1.plot(mean['concentration'], mean['tau'], 'b.-', lw='2', markersize=20)
ax2.plot(mean['concentration'], mean['A'], 'g.-', lw='2', markersize=20)

ax1.set_xlabel('Glucose concentration (mmol)')
ax1.set_xlabel('Intralipid concentration (%)')
ax1.set_ylabel('Lifetime (ms)', color='b')
ax2.set_ylabel('Amplitude (A.U.)', color='g')

ax1.grid()
plt.tight_layout()
ax1.ticklabel_format(useOffset=False, axis='y')
ax2.ticklabel_format(useOffset=False, axis='y')

plt.tight_layout()
# plt.title(df['chip'][0])
# plt.savefig("Plots/together/" + sample + '_T12_intralipidConc.png', dpi=500)

# Plot sensor data

In [ ]:
# Create plot of lifetime vs time
fig, ax = plt.subplots()
ax.plot(df['datetime'], df['tempC'], '-')
# ax.plot(df['datetime'], df['humidity'], '-')
# ax.plot(df['datetime'], df['tempC'], '-')

# format the ticks
ax.xaxis.set_major_locator(mdates.MinuteLocator(byminute=np.arange(0, 60, flush_time)))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
fig.autofmt_xdate()  # Rotate slightly

ax.xaxis.grid(True, which="major")
ax.yaxis.grid()
plt.tight_layout()
plt.ticklabel_format(useOffset=False, axis='y')

plt.xlabel('Time (H:M)')
plt.ylabel('Lifetime (ms)')
plt.legend()
# plt.savefig("Data/" + str(timestamp) + '/lifetimeVsTime.png', dpi=500)

In [ ]:
# Create plot of lifetime vs time
fig, ax = plt.subplots()
ax.plot(df['datetime'], df['thermocouple_in'], '-')
ax.plot(df['datetime'], df['thermocouple_out'], '-')
# ax.plot(df['datetime'], df['tempC'], '-')

# format the ticks
ax.xaxis.set_major_locator(mdates.MinuteLocator(byminute=np.arange(0, 60, flush_time)))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
fig.autofmt_xdate()  # Rotate slightly

ax.xaxis.grid(True, which="major")
ax.yaxis.grid()
plt.tight_layout()
plt.ticklabel_format(useOffset=False, axis='y')

plt.xlabel('Time (H:M)')
plt.ylabel('Lifetime (ms)')
plt.legend()
# plt.savefig("Data/" + str(timestamp) + '/lifetimeVsTime.png', dpi=500)

In [ ]:
r = df.set_index('delta').rolling(window=100)

fig, ax = plt.subplots()

ax.plot(r['thermocouple_in'].mean(), '-', lw=2)
ax.plot(r['thermocouple_out'].mean(), '-', lw=2)

ax.xaxis.grid(True, which="major")
ax.yaxis.grid()
# plt.xticks(np.arange(0, max(df['delta'])+1, flush_time))
plt.xticks(np.arange(0, max(df['delta'])+1, 1))
# plt.tight_layout()
plt.ticklabel_format(useOffset=False, axis='y')

plt.xlabel('Time since start of experiment (Mins)')
plt.ylabel('Temperature (degrees)')
plt.legend()
# plt.savefig("../Data/RollingMean.png", dpi=600)